### configuração inicial e carregamento de dados

In [1]:
import pandas as pd
import os
import sys
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

#### configuração inicial e importações

In [7]:
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..', '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"a raiz do projeto é: {project_root}")
print(f"a pasta 'ferramentas' existe em project_root? {os.path.exists(os.path.join(project_root, 'ferramentas'))}")

from ferramentas.leitura_e_escrita import ler_dados_brutos, salvar_dados_processados
from ferramentas.limpeza_geral import remover_linhas_vazias, converter_para_numerico, padronizar_nomes_colunas
from ferramentas.calculos_lucro import calcular_lucro_pesca
from ferramentas.tratamento_colunas import extrair_hora_inicio, converter_hora_para_periodo, processar_localizacao

caminho_base_stardew_data = os.path.join(project_root, 'jogos', 'stardew-valley')
caminho_dados_brutos_pesca = os.path.join(caminho_base_stardew_data, 'dados-brutos', 'stardew-valley-pesca.csv')
caminho_dados_processados_pesca = os.path.join(caminho_base_stardew_data, 'dados-processados', 'stardew-valley-pesca-calculado.csv')
caminho_pasta_graficos_pesca = os.path.join(project_root, 'jogos', 'stardew-valley', 'gráficos')
caminho_banco_dados = os.path.join(project_root, 'analise_jogos.db')

os.makedirs(caminho_pasta_graficos_pesca, exist_ok=True)

print(f"caminho completo para dados brutos de pesca: {caminho_dados_brutos_pesca}")
print(f"caminho completo para dados processados de pesca: {caminho_dados_processados_pesca}")
print(f"caminho completo para a pasta de gráficos de pesca: {caminho_pasta_graficos_pesca}")
print(f"caminho completo para o banco de dados: {caminho_banco_dados}")

a raiz do projeto é: c:\Users\giovanna\Documents\github\analise-jogos
a pasta 'ferramentas' existe em project_root? True
caminho completo para dados brutos de pesca: c:\Users\giovanna\Documents\github\analise-jogos\jogos\stardew-valley\dados-brutos\stardew-valley-pesca.csv
caminho completo para dados processados de pesca: c:\Users\giovanna\Documents\github\analise-jogos\jogos\stardew-valley\dados-processados\stardew-valley-pesca-calculado.csv
caminho completo para a pasta de gráficos de pesca: c:\Users\giovanna\Documents\github\analise-jogos\jogos\stardew-valley\gráficos
caminho completo para o banco de dados: c:\Users\giovanna\Documents\github\analise-jogos\analise_jogos.db


#### carregar dados brutos

In [3]:
if not os.path.exists(caminho_dados_brutos_pesca):
    print(f"erro: o arquivo '{caminho_dados_brutos_pesca} não foi encontrado.")
    df_pesca_bruto = pd.DataFrame()
else:
    print(f"\ncarregando dados brutos de pesca de: {caminho_dados_brutos_pesca}")
    df_pesca_bruto = ler_dados_brutos(caminho_dados_brutos_pesca)
    print(f"dados brutos de pesca carregados com sucesso!")


carregando dados brutos de pesca de: c:\Users\giovanna\Documents\github\analise-jogos\jogos\stardew-valley\dados-brutos\stardew-valley-pesca.csv
arquivo CSV 'c:\Users\giovanna\Documents\github\analise-jogos\jogos\stardew-valley\dados-brutos\stardew-valley-pesca.csv' lido com sucesso.
dados brutos de pesca carregados com sucesso!


#### exploração inicial dos dados brutos

In [50]:
print("--- primeiras 5 linhas do dataframe ---")
print(df_pesca_bruto.head())

--- primeiras 5 linhas do dataframe ---
      peixe  saúde restaurada (normal)  saúde restaurada (prata)  \
0    baiacu                        0.0                       0.0   
1   anchova                       13.0                      18.0   
2      atum                       38.0                      53.0   
3  sardinha                       13.0                      18.0   
4     brema                       13.0                      18.0   

   saúde restaurada (ouro)  saúde restaurada (irídio)  \
0                      0.0                        0.0   
1                     23.0                       33.0   
2                     68.0                       98.0   
3                     23.0                       33.0   
4                     23.0                       33.0   

   energia restaurada (normal)  energia restaurada (prata)  \
0                          0.0                         0.0   
1                          5.0                         8.0   
2                     

In [51]:
print("\n--- informações sobre o dataframe ---")
print(df_pesca_bruto.info())


--- informações sobre o dataframe ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   peixe                        48 non-null     object 
 1   saúde restaurada (normal)    46 non-null     float64
 2   saúde restaurada (prata)     46 non-null     float64
 3   saúde restaurada (ouro)      46 non-null     float64
 4   saúde restaurada (irídio)    46 non-null     float64
 5   energia restaurada (normal)  46 non-null     float64
 6   energia restaurada (prata)   46 non-null     float64
 7   energia restaurada (ouro)    46 non-null     float64
 8   energia restaurada (irídio)  46 non-null     float64
 9   preço de venda (normal)      48 non-null     int64  
 10  preço de venda (prata)       48 non-null     int64  
 11  preço de venda (ouro)        48 non-null     int64  
 12  preço de venda (íridio)      48 non-null 

In [52]:
print("\n--- dimensões do dataframe ---")
print(f"o dataframe possui {df_pesca_bruto.shape[0]} linhas e {df_pesca_bruto.shape[1]} colunas")


--- dimensões do dataframe ---
o dataframe possui 48 linhas e 17 colunas


In [53]:
print("\n--- nome das colunas ---")
print(df_pesca_bruto.columns)


--- nome das colunas ---
Index(['peixe', 'saúde restaurada (normal)', 'saúde restaurada (prata)',
       'saúde restaurada (ouro)', 'saúde restaurada (irídio)',
       'energia restaurada (normal)', 'energia restaurada (prata)',
       'energia restaurada (ouro)', 'energia restaurada (irídio)',
       'preço de venda (normal)', 'preço de venda (prata)',
       'preço de venda (ouro)', 'preço de venda (íridio)', 'localização',
       'horário', 'estação', 'clima'],
      dtype='object')


In [54]:
print("\n--- contagem de valores nulos ---")
print(df_pesca_bruto.isnull().sum())


--- contagem de valores nulos ---
peixe                          0
saúde restaurada (normal)      2
saúde restaurada (prata)       2
saúde restaurada (ouro)        2
saúde restaurada (irídio)      2
energia restaurada (normal)    2
energia restaurada (prata)     2
energia restaurada (ouro)      2
energia restaurada (irídio)    2
preço de venda (normal)        0
preço de venda (prata)         0
preço de venda (ouro)          0
preço de venda (íridio)        0
localização                    0
horário                        0
estação                        0
clima                          0
dtype: int64


In [55]:
print("\n--- resumo estatístico das colunas numéricas ---")
print(df_pesca_bruto.describe())


--- resumo estatístico das colunas numéricas ---
       saúde restaurada (normal)  saúde restaurada (prata)  \
count                  46.000000                 46.000000   
mean                   28.956522                 40.456522   
std                    20.008839                 28.007703   
min                     0.000000                  0.000000   
25%                    25.000000                 35.000000   
50%                    25.000000                 35.000000   
75%                    38.000000                 53.000000   
max                   125.000000                175.000000   

       saúde restaurada (ouro)  saúde restaurada (irídio)  \
count                46.000000                  46.000000   
mean                 51.956522                  64.978261   
std                  36.006763                  52.257690   
min                   0.000000                   0.000000   
25%                  45.000000                  33.000000   
50%                  45.0

In [56]:
print("\n--- contagem de peixes por localização ---")
print(df_pesca_bruto['localização'].value_counts())


--- contagem de peixes por localização ---
localização
oceano                                                   15
rio                                                       8
mina                                                      4
lago da montanha                                          3
rio, lago da floresta                                     3
o deserto                                                 2
lago da montanha, lagoa do bosque secreto, os esgotos     1
rio, lagoa do bosque secreto, pântano da bruxa            1
rio, lago da floresta, lago da montanha                   1
rio, lago da montanha                                     1
lago da montanha, lago da floresta                        1
oceano, lago da montanha                                  1
lagoa do bosque secreto                                   1
pântano da bruxa                                          1
covil dos insetos mutantes                                1
oceano da ilha gengibre                     

In [57]:
print("\n--- contagem de peixes por horário ---")
print(df_pesca_bruto['horário'].value_counts())


--- contagem de peixes por horário ---
horário
qualquer hora    20
6:00 – 19:00     12
18:00 – 2:00      3
6:00 – 20:00      3
6:00 – 11:00      2
12:00 – 16:00     1
12:00 – 2:00      1
6:00 – 00:00      1
16:00 – 2:00      1
6:00 – 13:00      1
22:00 – 2:00      1
6:00 – 14:00      1
9:00 – 2:00       1
Name: count, dtype: int64


In [58]:
print("\n--- contagem de peixes por estação ---")
print(df_pesca_bruto['estação'].value_counts())


--- contagem de peixes por estação ---
estação
todas                         18
outono, inverno                5
verão                          4
primavera, outono              4
verão, inverno                 4
inverno                        3
primavera, verão               2
verão, outono                  2
primavera, outono, inverno     1
outono                         1
primavera, inverno             1
verão, outono, inverno         1
primavera, verão, outono       1
primavera, verão, inverno      1
Name: count, dtype: int64


In [59]:
print("\n--- contagem de peixes por clima ---")
print(df_pesca_bruto['clima'].value_counts())


--- contagem de peixes por clima ---
clima
qualquer      40
chuva          5
sol            2
sol, vento     1
Name: count, dtype: int64


### limpeza e padronização de dados

In [5]:
print("\niniciando a limpeza dos dados de pesca...")

df_pesca_limpo = remover_linhas_vazias(df_pesca_bruto)
print("\nlinhas vazias removidas!")

colunas_numericas = ['saúde restaurada (normal)', 'saúde restaurada (prata)',
       'saúde restaurada (ouro)', 'saúde restaurada (irídio)',
       'energia restaurada (normal)', 'energia restaurada (prata)',
       'energia restaurada (ouro)', 'energia restaurada (irídio)',
       'preço de venda (normal)', 'preço de venda (prata)',
       'preço de venda (ouro)', 'preço de venda (íridio)']
for col in colunas_numericas:
    if col in df_pesca_limpo.columns:
        df_pesca_limpo = converter_para_numerico(df_pesca_limpo, col)
        print("\ncolunas numéricas convertidas!")

df_pesca_limpo = padronizar_nomes_colunas(df_pesca_bruto)
print("\nnomes das colunas padronizados!")


iniciando a limpeza dos dados de pesca...
removidas 0 linhas vazias.

linhas vazias removidas!
aviso: coluna 's' não encontrada no DataFrame para conversão.
aviso: coluna 'a' não encontrada no DataFrame para conversão.
aviso: coluna 'ú' não encontrada no DataFrame para conversão.
aviso: coluna 'd' não encontrada no DataFrame para conversão.
aviso: coluna 'e' não encontrada no DataFrame para conversão.
aviso: coluna ' ' não encontrada no DataFrame para conversão.
aviso: coluna 'r' não encontrada no DataFrame para conversão.
aviso: coluna 'e' não encontrada no DataFrame para conversão.
aviso: coluna 's' não encontrada no DataFrame para conversão.
aviso: coluna 't' não encontrada no DataFrame para conversão.
aviso: coluna 'a' não encontrada no DataFrame para conversão.
aviso: coluna 'u' não encontrada no DataFrame para conversão.
aviso: coluna 'r' não encontrada no DataFrame para conversão.
aviso: coluna 'a' não encontrada no DataFrame para conversão.
aviso: coluna 'd' não encontrada no 

In [61]:
print("\n--- informações dataframe limpo ---")
print(df_pesca_limpo.info())
print(df_pesca_limpo.head())


--- informações dataframe limpo ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   peixe                      48 non-null     object 
 1   saude_restaurada_normal    46 non-null     float64
 2   saude_restaurada_prata     46 non-null     float64
 3   saude_restaurada_ouro      46 non-null     float64
 4   saude_restaurada_iridio    46 non-null     float64
 5   energia_restaurada_normal  46 non-null     float64
 6   energia_restaurada_prata   46 non-null     float64
 7   energia_restaurada_ouro    46 non-null     float64
 8   energia_restaurada_iridio  46 non-null     float64
 9   preco_de_venda_normal      48 non-null     int64  
 10  preco_de_venda_prata       48 non-null     int64  
 11  preco_de_venda_ouro        48 non-null     int64  
 12  preco_de_venda_iridio      48 non-null     int64  
 13  localizacao    

### cálculos de lucro e criação de novas colunas

In [11]:
print("\nrealizando cálculos específicos para peixes...")

colunas_venda_pesca = ['preco_de_venda_normal', 'preco_de_venda_prata', 'preco_de_venda_ouro', 'preco_de_venda_iridio']
df_pesca_calculado = calcular_lucro_pesca(df_pesca_limpo, colunas_venda=colunas_venda_pesca)

print("cálculos de pesca concluídos. primeiras 5 linhas e colunas adicionadas:")
print(df_pesca_calculado.head())
print(df_pesca_calculado.columns)


realizando cálculos específicos para peixes...
cálculos de pesca concluídos. primeiras 5 linhas e colunas adicionadas:
      peixe  saude_restaurada_normal  saude_restaurada_prata  \
0    baiacu                      0.0                     0.0   
1   anchova                     13.0                    18.0   
2      atum                     38.0                    53.0   
3  sardinha                     13.0                    18.0   
4     brema                     13.0                    18.0   

   saude_restaurada_ouro  saude_restaurada_iridio  energia_restaurada_normal  \
0                    0.0                      0.0                        0.0   
1                   23.0                     33.0                        5.0   
2                   68.0                     98.0                       17.0   
3                   23.0                     33.0                        5.0   
4                   23.0                     33.0                        5.0   

   energia_res

#### tratamento das colunas

In [ ]:
df_pesca_calculado['hora_numerica'] = df_pesca_calculado['horario'].apply(extrair_hora_inicio)
df_pesca_calculado['periodo_do_dia'] = df_pesca_calculado['hora_numerica'].apply(converter_hora_para_periodo)

df_pesca_calculado = processar_localizacao(df_pesca_calculado)



print("dataframe com as novas colunas de horário e localização:")
print(df_pesca_calculado[['horario', 'hora_numerica', 'periodo_do_dia', 'localizacao_individual']].head())

dataframe com as novas colunas de horário e localização:
         horario  hora_numerica periodo_do_dia localizacao_individual
0  12:00 – 16:00            NaN  qualquer hora                 oceano
1  qualquer hora            NaN  qualquer hora                 oceano
2   6:00 – 19:00            NaN  qualquer hora                 oceano
3   6:00 – 19:00            NaN  qualquer hora                 oceano
4   18:00 – 2:00            NaN  qualquer hora                    rio
